In [ ]:
!pip install gradio -q
!pip install wandb --upgrade -q
!pip install -q git+https://github.com/huggingface/transformers.git

     |████████████████████████████████| 2.9 MB 27.1 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 253 kB 42.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 211 kB 60.2 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 2.0 MB 43.8 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 65.2 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 58 kB 2.8 MB/s 
     |████████████████████████████████| 10.9 MB 19.8 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 856 kB 66.2 MB/s 
     |████████████████████████████████| 61 kB 409 kB/s 
     |███████████████████████

In [ ]:
import gradio as gr
import librosa
import soundfile as sf
import torch
from transformers import Wav2Vec2ForMaskedLM, Wav2Vec2Tokenizer

In [ ]:
import torch
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from tqdm import tqdm
#wandb.init(project="Audio2Text", entity="raghavadurs", id="asr_5")
wandb.init()
config = wandb.config
config.learning_rate = 0.01

wandb: Currently logged in as: korean (use `wandb login --relogin` to force relogin)


In [ ]:
# load wav2vec2 tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
# processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
model = Wav2Vec2ForMaskedLM.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

#define speech to text function
def asr_transcribe(audio_file):
  transcript = ""

  # Stream over 20seconds chunks
  stream = librosa.stream(
      audio_file.name, block_length=20, frame_length=16000, hop_length=16000
  )

  for speech in stream:
    if len(speech.shape) > 1:
      speech = speech[:, 0] + speech[:, 1]

    input_values = tokenizer(speech, return_tensors="pt").input_values
    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    transcript += transcription.lower() + " "

  return transcript

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/162 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:752: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1603: FutureWarning: The class `Wav2Vec2ForMaskedLM` is deprecated. Please use `Wav2Vec2ForCTC` instead.
  "The class `Wav2Vec2ForMaskedLM` is deprecated. Please use `Wav2Vec2ForCTC` instead.", FutureWarning
Some weights of Wav2Vec2ForMaskedLM were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
gradio_asr = gr.Interface(
    fn=asr_transcribe,
    title="Speech To Text with Wav2Vec2 데모",
    description="오디오 클립을 넣으면, 번역해줘요!!",
    inputs=gr.inputs.Audio(label="오디오 파일 업로드", type="file"),
    outputs=gr.outputs.Textbox(label="자동번역"),
)

In [ ]:
gradio_asr.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://26221.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f4cec174d50>,
 'http://127.0.0.1:7861/',
 'https://26221.gradio.app')